# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [8]:
import pandas as pd
import statsmodels.api as sm
import patsy as pt
import numpy as np

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [4]:
    # Remover a coluna 'data_ref' se não for necessária no modelo
    df = df.drop(columns=['data_ref'])

    # Converter as colunas categóricas em dummies (variáveis indicadoras)
    df_dummies = pd.get_dummies(df, drop_first=True)

    # Verificar o novo DataFrame dummificado
    print(df_dummies.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     15000 non-null  int64  
 1   index                          15000 non-null  int64  
 2   posse_de_veiculo               15000 non-null  bool   
 3   posse_de_imovel                15000 non-null  bool   
 4   qtd_filhos                     15000 non-null  int64  
 5   idade                          15000 non-null  int64  
 6   tempo_emprego                  12466 non-null  float64
 7   qt_pessoas_residencia          15000 non-null  float64
 8   mau                            15000 non-null  bool   
 9   renda                          15000 non-null  float64
 10  sexo_M                         15000 non-null  bool   
 11  tipo_renda_Bolsista            15000 non-null  bool   
 12  tipo_renda_Empresário          15000 non-null 

In [5]:
# Listar todas as colunas booleanas
bool_columns = df_dummies.select_dtypes(include='bool').columns

# Converter as colunas booleanas para 0s e 1s
df_dummies[bool_columns] = df_dummies[bool_columns].astype(int)

# Verificar o novo DataFrame para garantir que as colunas booleanas foram convertidas
print(df_dummies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     15000 non-null  int64  
 1   index                          15000 non-null  int64  
 2   posse_de_veiculo               15000 non-null  int32  
 3   posse_de_imovel                15000 non-null  int32  
 4   qtd_filhos                     15000 non-null  int64  
 5   idade                          15000 non-null  int64  
 6   tempo_emprego                  12466 non-null  float64
 7   qt_pessoas_residencia          15000 non-null  float64
 8   mau                            15000 non-null  int32  
 9   renda                          15000 non-null  float64
 10  sexo_M                         15000 non-null  int32  
 11  tipo_renda_Bolsista            15000 non-null  int32  
 12  tipo_renda_Empresário          15000 non-null 

In [9]:
df['log_renda'] = np.log(df['renda'])

formula = 'log_renda ~ Q("sexo") + Q("posse_de_veiculo") + Q("posse_de_imovel") + Q("qtd_filhos") + \
           Q("tipo_renda") + Q("educacao") + Q("estado_civil") + Q("tipo_residencia") + Q("idade") + \
           Q("tempo_emprego") + Q("qt_pessoas_residencia")'

# Criar as matrizes de design com Patsy
y, X = pt.dmatrices(formula, data=df, return_type='dataframe')

# Ajustar o modelo inicial
modelo_inicial = sm.OLS(y, sm.add_constant(X)).fit()

# Resumo do modelo
print(modelo_inicial.summary())

                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.242
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     165.3
Date:                Wed, 25 Sep 2024   Prob (F-statistic):               0.00
Time:                        23:47:12   Log-Likelihood:                -13204.
No. Observations:               12466   AIC:                         2.646e+04
Df Residuals:                   12441   BIC:                         2.664e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

In [10]:
# Selecionar as variáveis com p-valor menor que 0.05
significativas = modelo_inicial.pvalues[modelo_inicial.pvalues < 0.05].index.tolist()

# Ajustar o modelo novamente com as variáveis significativas
X_significativas = X[significativas]
modelo_significativo = sm.OLS(y, sm.add_constant(X_significativas)).fit()

# Resumo do modelo ajustado
print(modelo_significativo.summary())

                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.237
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     551.9
Date:                Wed, 25 Sep 2024   Prob (F-statistic):               0.00
Time:                        23:47:57   Log-Likelihood:                -13246.
No. Observations:               12466   AIC:                         2.651e+04
Df Residuals:                   12458   BIC:                         2.657e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In